# 🧠 Deep Sector Rotation Swing Trading Notebook

This notebook implements a weekly ETF sector rotation strategy inspired by the paper:

**"Deep Sector Rotation Swing Trading"** (Bock & Maewal, SSRN #4280640)

---

### 📌 Strategy Summary:
- Trades once per week (Buy on Monday open, Sell on Friday close)
- Uses a deep learning model to predict next-week returns for selected ETFs
- Selects high-confidence trades using Monte Carlo Dropout
- Allocates capital selectively based on prediction strength

---

### 🔧 Key Components:
- Weekly ETF price data (e.g., XLK, XLF, XLV…)
- Rolling technical and macro features
- Multi-output regression model
- Weekly backtest with position logging and performance metrics


In [47]:
import os
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta

# ETF list
etf_list = [
    'XLK', 'XLF', 'XLV', 'XLE', 'XLI', 'XLY', 'XLP', 'XLRE', 'XLU', 'XLB', 'XLC',
    'SOXX', 'SH', 'DOG', 'RWM', 'ITA', 'JETS'
]

# Date range
end_date = datetime.today().strftime('%Y-%m-%d')
start_date = (datetime.today() - timedelta(weeks=5*52)).strftime('%Y-%m-%d')

print(f"📅 Downloading data from {start_date} to {end_date}")

# Ensure dataset/ exists
dataset_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'dataset'))
if not os.path.isdir(dataset_path):
    raise FileNotFoundError(f"🚫 'dataset/' folder not found at {dataset_path}")

# Containers
adjclose_data, volume_data, high_data, low_data = {}, {}, {}, {}

# Download each ETF
for symbol in etf_list:
    print(f"⬇️ Downloading {symbol}...")
    data = yf.download(symbol, start=start_date, end=end_date, interval='1wk', auto_adjust=False)
    if not data.empty:
        adjclose_data[symbol] = data[['Adj Close']].rename(columns={'Adj Close': symbol})
        volume_data[symbol] = data[['Volume']].rename(columns={'Volume': symbol})
        high_data[symbol] = data[['High']].rename(columns={'High': symbol})
        low_data[symbol] = data[['Low']].rename(columns={'Low': symbol})

# Merge and clean
def combine_and_save(data_dict, filename):
    df = pd.concat(data_dict.values(), axis=1)
    df = df.apply(pd.to_numeric, errors='coerce')
    df.index = pd.to_datetime(df.index, errors='coerce')
    df = df[~df.index.duplicated(keep='first')].sort_index()
    df.dropna(axis=0, how='all', inplace=True)
    path = os.path.join(dataset_path, filename)
    df.to_csv(path)
    print(f"✅ Saved: {filename}")
    return df

# Save all
price_df = combine_and_save(adjclose_data, 'etf_prices_weekly.csv')
volume_df = combine_and_save(volume_data, 'etf_volume_weekly.csv')
high_df = combine_and_save(high_data, 'etf_high_weekly.csv')
low_df = combine_and_save(low_data, 'etf_low_weekly.csv')

# Preview
price_df.head()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

📅 Downloading data from 2020-04-19 to 2025-04-13
⬇️ Downloading XLK...
⬇️ Downloading XLF...
⬇️ Downloading XLV...
⬇️ Downloading XLE...
⬇️ Downloading XLI...
⬇️ Downloading XLY...
⬇️ Downloading XLP...
⬇️ Downloading XLRE...



[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

⬇️ Downloading XLU...
⬇️ Downloading XLB...
⬇️ Downloading XLC...
⬇️ Downloading SOXX...
⬇️ Downloading SH...
⬇️ Downloading DOG...
⬇️ Downloading RWM...
⬇️ Downloading ITA...
⬇️ Downloading JETS...
✅ Saved: etf_prices_weekly.csv
✅ Saved: etf_volume_weekly.csv
✅ Saved: etf_high_weekly.csv
✅ Saved: etf_low_weekly.csv


Price,XLK,XLF,XLV,XLE,XLI,XLY,XLP,XLRE,XLU,XLB,XLC,SOXX,SH,DOG,RWM,ITA,JETS
Ticker,XLK,XLF,XLV,XLE,XLI,XLY,XLP,XLRE,XLU,XLB,XLC,SOXX,SH,DOG,RWM,ITA,JETS
Date,,,,,,,,,,,,,,,,,
2020-04-20,84.995071,19.800924,92.707420,28.168427,56.930317,107.774223,51.462582,27.985085,49.870876,45.131828,46.481499,73.612915,92.778679,47.638866,39.331772,70.986526,13.265752
2020-04-27,85.254005,20.092384,90.433426,29.169197,57.577572,107.764595,50.507133,27.815889,47.784370,45.973644,47.360310,71.331749,92.704010,47.620911,38.178120,71.822174,13.990603
2020-05-04,90.892860,20.301870,91.893959,31.553186,58.354282,112.584427,50.945404,28.213499,48.023796,47.503384,49.538235,76.902237,89.493156,46.390884,36.040722,72.466812,13.543777
2020-05-11,89.636589,19.172466,92.827583,29.283121,54.979321,110.875427,50.305538,26.208521,46.869381,46.046055,48.955544,73.753258,91.397270,47.549080,38.035027,67.185555,11.915346
2020-05-18,92.532707,19.964870,92.088081,31.309097,59.029266,116.396111,50.375648,27.663614,48.297443,47.910709,51.305405,78.060364,88.373100,45.879112,35.083801,73.063698,13.603354


### 🌐 Weekly Macro Indicator Download

This section downloads weekly data for key macroeconomic signals that are used as input features for the model:

| Indicator        | Source Symbol | Description |
|------------------|---------------|-------------|
| **VIX**          | `^VIX`        | CBOE Volatility Index (market fear gauge) |
| **10Y Yield**    | `^TNX`        | 10-Year U.S. Treasury yield (interest rate proxy) |
| **USD Index**    | `DX-Y.NYB`    | Strength of the U.S. dollar |
| **Crude Oil**    | `CL=F`        | WTI Crude Oil futures price |

All indicators are:
- Downloaded at **weekly frequency** using Yahoo Finance
- Aligned on the same date index as the ETF data
- The 10-year yield is converted to a % by multiplying by `0.1`


In [48]:
import os
import yfinance as yf
import pandas as pd

# Macro indicator tickers on Yahoo Finance
macro_tickers = {
    'VIX': '^VIX',               # Volatility Index
    '10Y_Yield': '^TNX',         # 10-Year Treasury Yield (multiply by 0.1)
    'USD_Index': 'DX-Y.NYB',     # U.S. Dollar Index
    'WTI_Crude': 'CL=F'          # Crude Oil (WTI)
}

# Date range matching your ETF backtest period
start_date = (pd.Timestamp.today() - pd.DateOffset(years=5)).strftime('%Y-%m-%d')
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

# Download weekly data
macro_data = {}
for name, ticker in macro_tickers.items():
    print(f"Downloading {name} ({ticker})...")
    data = yf.download(ticker, start=start_date, end=end_date, interval='1wk', auto_adjust=False)
    macro_data[name] = data[['Close']].rename(columns={'Close': name})

# Combine all macro indicators into one DataFrame
macro_df = pd.concat(macro_data.values(), axis=1)

# Fix 10Y yield scale
if '10Y_Yield' in macro_df.columns:
    macro_df['10Y_Yield'] = macro_df['10Y_Yield'] * 0.1

# Drop missing rows
macro_df.dropna(inplace=True)

# Save to CSV
macro_save_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'dataset', 'macro_indicators_weekly.csv'))
macro_df.to_csv(macro_save_path)
print(f"✅ Macro indicators saved to: {macro_save_path}")

# Preview

macro_df = macro_df.apply(pd.to_numeric, errors='coerce')
macro_df.index = pd.to_datetime(macro_df.index)
macro_df = macro_df[~macro_df.index.duplicated(keep='first')]
macro_df.sort_index(inplace=True)
macro_df.head()



[*********************100%***********************]  1 of 1 completed


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

✅ Macro indicators saved to: d:\CodingWorks\my_MLQT_project\Stock-Prediction-Models\dataset\macro_indicators_weekly.csv


Price,VIX,10Y_Yield,USD_Index,WTI_Crude
Ticker,^VIX,^TNX,DX-Y.NYB,CL=F
Date,,,,
2020-04-13,38.150002,0.0654,99.779999,18.270000
2020-04-20,35.930000,0.0596,100.379997,16.940001
2020-04-27,37.189999,0.0642,98.800003,19.780001
2020-05-04,27.980000,0.0682,99.730003,24.740000
2020-05-11,31.889999,0.0640,100.400002,29.430000


### 🧠 Feature Engineering

This section prepares input features for the machine learning model.

#### 📈 ETF-Specific Features:
For each ETF, we will compute:
- **1-week return**: Short-term price movement
- **3-week return**: Medium-term trend
- **6-week return**: Momentum across a longer window
- **Streak**: Number of consecutive up weeks

#### 🌐 Macro Indicators:
From the macro_df, we already have:
- **VIX**
- **10Y Treasury Yield**
- **USD Index**
- **Crude Oil Price**

These will be aligned with the ETF data by date and merged in.

#### 📦 Resulting Feature Matrix:
For each ETF on each week:
- One row = a snapshot of that ETF and macro environment
- Target = the **next week's return** for that ETF


In [50]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# === Load datasets ===
price_df = pd.read_csv('../dataset/etf_prices_weekly.csv', index_col=0)
volume_df = pd.read_csv('../dataset/etf_volume_weekly.csv', index_col=0)
macro_df = pd.read_csv('../dataset/macro_indicators_weekly.csv', index_col=0)
high_df = pd.read_csv('../dataset/etf_high_weekly.csv', index_col=0)
low_df = pd.read_csv('../dataset/etf_low_weekly.csv', index_col=0)

# === Clean and convert: force numeric and datetime, in-place ===
for df in [price_df, volume_df, macro_df, high_df, low_df]:
    df.index = pd.to_datetime(df.index, errors='coerce')
    df[:] = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=0, how='all', inplace=True)
    df.dropna(axis=1, how='all', inplace=True)
    df.sort_index(inplace=True)
    df.dropna(inplace=True)

# === Feature generation ===
feature_rows = []

for symbol in price_df.columns:
    returns_1w = price_df[symbol].pct_change(1)
    returns_3w = price_df[symbol].pct_change(3)
    returns_6w = price_df[symbol].pct_change(6)
    streak = (price_df[symbol].pct_change(1) > 0).astype(int).rolling(3).sum()

    # Volume feature
    volume_series = pd.to_numeric(volume_df[symbol], errors='coerce').replace(0, np.nan)
    log_volume = np.log(volume_series)
    log_volume_norm = log_volume / log_volume.rolling(window=5).mean()

    # Shock Amplify Rate features
    high = high_df[symbol]
    low = low_df[symbol]
    close = price_df[symbol]
    shock_amplify = (high - low) / close
    shock_amplify_1w = shock_amplify.shift(1)
    shock_amplify_3w = shock_amplify.rolling(window=3).mean()

    for i in range(6, len(price_df) - 1):
        date = price_df.index[i]

        try:
            nearest_macro_index = macro_df.index.get_indexer([date], method='nearest')[0]
            macro_row = macro_df.iloc[nearest_macro_index]
        except (KeyError, IndexError, ValueError):
            continue

        row = {
            'Date': date,
            'ETF': symbol,
            'Return_1w': returns_1w.iloc[i],
            'Return_3w': returns_3w.iloc[i],
            'Return_6w': returns_6w.iloc[i],
            'Streak_Up': streak.iloc[i],
            'LogVolumeNorm': log_volume_norm.iloc[i],
            'Shock_Amplify': shock_amplify.iloc[i],
            'Shock_Amplify_1w': shock_amplify_1w.iloc[i],
            'Shock_Amplify_3w': shock_amplify_3w.iloc[i],
            'Target_Next_Week_Return': price_df[symbol].pct_change(1).shift(-1).iloc[i]
        }

        for col in macro_df.columns:
            row[col] = macro_row[col]

        feature_rows.append(row)

# === Final DataFrame ===
feature_df = pd.DataFrame(feature_rows)
feature_df.dropna(inplace=True)

# Preview
feature_df.head()



C:\Users\user\AppData\Local\Temp\ipykernel_46752\2343034369.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index, errors='coerce')
C:\Users\user\AppData\Local\Temp\ipykernel_46752\2343034369.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index, errors='coerce')
C:\Users\user\AppData\Local\Temp\ipykernel_46752\2343034369.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df.index = pd.to_datetime(df.index, errors='coerce')
C:\Users\user\AppData\Local\Temp\ipykernel_46752\2343034369.py:15: UserWarning: Co

,Date,ETF,Return_1w,Return_3w,Return_6w,Streak_Up,LogVolumeNorm,Shock_Amplify,Shock_Amplify_1w,Shock_Amplify_3w,Target_Next_Week_Return,VIX,10Y_Yield,USD_Index,WTI_Crude
0,2020-06-01,XLK,0.035113,0.084947,0.144195,3.0,0.998299,0.047506,0.044597,0.040463,-0.019722,24.520000,0.0904,96.940002,39.549999
1,2020-06-08,XLK,-0.019722,0.030262,0.118223,2.0,1.008045,0.075630,0.047506,0.055911,0.028468,36.090000,0.0699,97.320000,36.259998
2,2020-06-15,XLK,0.028468,0.043585,0.078709,2.0,1.001950,0.071700,0.075630,0.064945,-0.006847,35.119999,0.0697,97.620003,39.750000
3,2020-06-22,XLK,-0.006847,0.001282,0.086338,1.0,1.000995,0.047240,0.071700,0.064857,0.039592,34.730000,0.0636,97.500000,38.490002
4,2020-06-29,XLK,0.039592,0.061866,0.094001,2.0,0.978754,0.058381,0.047240,0.059107,0.026693,27.680000,0.0669,97.320000,40.650002
